In [2]:
%load_ext autoreload
%autoreload 2
import os
from scipy.misc import imread, imresize
import tensorflow as tf
import numpy as np

from scipy.misc import imread
from src.utils import preprocess_image, deprocess_image, load_image
import matplotlib.pyplot as plt

%matplotlib inline

def get_isession():
    """Create a session that dynamically allocates memory."""
    # See: https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    # session = tf.Session(config=config)
    session = tf.InteractiveSession(config=config)
    return session

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from classifiers.squeezenet import SqueezeNet

## Content Loss
A simple l2_norm could do the job (image - target), but it will make more sense to convert it from: $R^{1 \times H_l \times W_l \times C_l}$ to $R^{C_l \times N_l}$, where $\begin{equation} N_l = H_l \times W_l \end{equation}$, $l$ refers to layer number. The unrolled version of the image will be usefull for the style loss later

In [3]:
def content_loss(weight, a_C, a_G):
    """
    Computes the content cost
    
    Arguments:
    weight -- scalar constant we multiply the content_loss by
    a_C -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing content of the image C 
    a_G -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing content of the image G
    
    Returns: 
    J_content -- scalar that you compute using equation 1 above.
    """
    
    # Retrieve dimensions from a_G (≈1 line)
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    
    # Reshape a_C and a_G (≈2 lines)
    a_C_unrolled = tf.reshape(tf.transpose(a_C, perm=[3,1,2,0]), shape=[n_C, -1])
    a_G_unrolled = tf.reshape(tf.transpose(a_G, perm=[3,1,2,0]), shape=[n_C, -1])
    
    # compute the cost with tensorflow (≈1 line)
    J_content = weight/(4 * n_H * n_W * n_C) * tf.reduce_sum((a_C_unrolled - a_G_unrolled) ** 2)
    
    return J_content

In [7]:
tf.reset_default_graph()

with tf.Session() as test:
    tf.set_random_seed(1)
    a_C = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    a_G = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    J_content = content_loss(1.0, a_C, a_G)
    print("J_content = " + str(J_content.eval()))

J_content = 6.7655935


## Style Loss
For this the Gram Matrix is used as a way of measuring the statistical similarity between the current image and the target image. The result dimension of the Gram Matrix is $C_l \times C_l$

In [4]:
def gram_matrix(A):
    """
    Argument:
    A -- matrix of shape (n_C, n_H*n_W)
    
    Returns:
    GA -- Gram matrix of A, of shape (n_C, n_C)
    """

    return A @ tf.transpose(A)

In [5]:
def layer_style_loss(weight, a_S, a_G):
    """
    Arguments:
    weight -- scalar constant we multiply the content_loss by
    a_S -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing style of the image S 
    a_G -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing style of the image G
    
    Returns: 
    J_style_layer -- tensor representing a scalar value, style cost defined above by equation (2)
    """
    
    # Retrieve dimensions from a_G (≈1 line)
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    
    # Reshape the images to have them of shape (n_C, n_H*n_W) (≈2 lines)
    a_S = tf.reshape(tf.transpose(a_S, perm=[3,1,2,0]), shape=[n_C, -1])
    a_G = tf.reshape(tf.transpose(a_G, perm=[3,1,2,0]), shape=[n_C, -1])

    # Computing gram_matrices for both images S and G (≈2 lines)
    GS = gram_matrix(a_S)
    GG = gram_matrix(a_G)

    # Computing the loss (≈1 line)
    J_style_layer = weight / (4 * n_C ** 2 * (n_H * n_W) ** 2) * tf.reduce_sum((GS - GG) ** 2) 
    
    return J_style_layer

In [9]:
tf.reset_default_graph()

with tf.Session() as test:
    tf.set_random_seed(1)
    a_S = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    a_G = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    J_style_layer = layer_style_loss(1.0, a_S, a_G)
    
    print("J_style_layer = " + str(J_style_layer.eval()))

J_style_layer = 9.190278


### Style layers
Multiple layers from the model will be merged for the final style loss, need to fine tune the weights

In [6]:
STYLE_LAYERS = [
    (1, 0.2),
    (4, 0.2),
    (6, 0.2),
    (7, 0.2)
]

In [ ]:
def style_loss(model, layers):
    J_style = 0
    for l, weight in layers:
        out = model.layers[l]
        

## Total-variation regularization
As stated in cs231n/assigment3, it's helpfull to add a term that penalizes wiggles in the pixel values. Compute it as the sum of squares of differences of each pair of pixels, in both dimensions for each input channels (RGB).

In [14]:
def tv_loss(weight, img):
    """
    Compute total variation loss.
    
    Inputs:
    - weight: scalar number to use for the tv_loss
    - img: Tensor of shape (1, H, W, 3) holding an input image.
    
    Returns:
    - loss: Tensor holding a scalar giving the total variation loss
      for img weighted by tv_weight.
    """
    
    # Yay! no loops :3
    h = img[:,:-1,:,:]
    w = img[:,:,:-1,:]
    h_shifted = img[:,1:,:,:]
    w_shifted = img[:,:,1:,:]
    
    return weight * tf.reduce_sum((h - h_shifted)[:,:,:-1,:]**2 + (w - w_shifted)[:,:-1,:,:] **2)

In [17]:
tf.reset_default_graph()

with tf.Session() as test:
    weight = 1e-2
    tf.set_random_seed(1)
    rgb = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    J_tv = tv_loss(weight, rgb)
    
    print("J_style_layer = " + str(J_tv.eval()))

J_style_layer = 16.563738


## Load Squeezenet
Use an interactive session, with gpu special options

In [6]:
MODEL_PATH = 'data/squeezenet.ckpt'

tf.reset_default_graph()
sess = get_isession()

if os.path.exists(MODEL_PATH):
    raise ValueError("You need to download SqueezeNet!")
model = SqueezeNet(save_path=MODEL_PATH, sess=sess)

INFO:tensorflow:Restoring parameters from data/squeezenet.ckpt
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

